In [ ]:
pip install -r requirements.txt

In [49]:
import numpy as np
import tensorflow as tf
import tempfile
import zipfile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [50]:
temp_dir = tempfile.TemporaryDirectory() 

In [51]:
with zipfile.ZipFile('dataset.zip', 'r') as zip: # Unzipping the dataset
    zip.extractall(temp_dir.name)

In [ ]:
classificador = Sequential() # Inicializa a rede neural
classificador.add(InputLayer(input_shape=(64, 64, 3))) # Camada de entrada da rede neural (64x64 pixels e 3 canais de cor)

classificador.add(Conv2D(32, (3, 3), activation='relu')) # Primeira camada de convolução com 32 filtros 3x3
classificador.add(BatchNormalization()) # Normalização dos valores de saída da camada de convolução 
classificador.add(MaxPooling2D(pool_size=(2, 2))) # Camada de pooling para redução da dimensionalidade

classificador.add(Conv2D(32, (3, 3), activation='relu')) # Segunda camada de convolução com 32 filtros 3x3  
classificador.add(BatchNormalization()) # Normalização dos valores de saída da camada de convolução
classificador.add(MaxPooling2D(pool_size=(2, 2))) # Camada de pooling para redução da dimensionalidade

classificador.add(Flatten())  # Camada de achatamento dos valores de saída da camada de pooling para entrada na rede neural densa 

classificador.add(Dense(units=128, activation='relu')) # Primeira camada densa com 128 neurônios
classificador.add(Dropout(0.2)) # Dropout de 20% para evitar overfitting
classificador.add(Dense(units=128, activation='relu')) # Segunda camada densa com 128 neurônios
classificador.add(Dropout(0.2)) # Dropout de 20% para evitar overfitting

classificador.add(Dense(units=1, activation='sigmoid')) # Camada de saída com 1 neurônio e função de ativação sigmoid

In [53]:
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Compilação do modelo com otimizador Adam, função de perda entropia cruzada binária e métrica de acurácia

In [54]:
gerador_treinamento = ImageDataGenerator(rescale=1./255, rotation_range=7, 
                                         horizontal_flip=True, shear_range=0.2, height_shift_range=0.07, zoom_range=0.2) # Gerador de imagens para o conjunto de treinamento

In [55]:
gerador_teste = ImageDataGenerator(rescale=1./255) # Gerador de imagens para o conjunto de teste

In [ ]:
base_treinamento = gerador_treinamento.flow_from_directory(temp_dir.name + '/dataset/training_set', 
                                                           target_size=(64, 64), batch_size=32, class_mode='binary') # Carregamento do conjunto de treinamento

In [ ]:
base_teste = gerador_teste.flow_from_directory(temp_dir.name + '/dataset/test_set', 
                                               target_size=(64, 64), batch_size=32, class_mode='binary') # Carregamento do conjunto de treinamento

In [ ]:
classificador.fit(base_treinamento, epochs=10, validation_data=base_teste) # Treinamento do modelo

In [67]:
imagem_teste = image.load_img(f'{temp_dir.name}/dataset/training_set/cachorro/dog.1922.jpg', target_size=(64, 64)) # Carregamento de uma imagem de teste

In [68]:
imagem_teste = image.img_to_array(imagem_teste) # Conversão da imagem para um array

In [69]:
imagem_teste /= 255 # Normalização dos valores dos pixels

In [73]:
imagem_teste = np.expand_dims(imagem_teste, axis=0) # Expansão da dimensão da imagem para ser compatível com a entrada da rede neural

In [ ]:
imagem_teste.shape

In [ ]:
previsao = classificador.predict(imagem_teste) # Previsão da imagem

In [79]:
previsao = previsao > 0.5 # Atribuição de True para a previsão maior que 0.5 e False para a previsão menor que 0.5

In [ ]:
previsao